In [2]:
import os
import pandas as pd
import pytz
import matplotlib.pyplot as plt
from datetime import datetime

# Para que los gráficos se vean en el notebook
%matplotlib inline


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Carpeta donde tienes los ficheros de sesiones
DATA_DIR = "./data/SPY"

# Zona horaria de España
SPAIN_TZ = pytz.timezone("Europe/Madrid")


In [ ]:
def read_session_file(file_path):
    """
    Lee un fichero Excel de IB, convierte time de UTC a España y calcula price medio
    """
    df = pd.read_excel(file_path)
    
    # Convertir columna 'time' a datetime tz-aware UTC
    df['time'] = pd.to_datetime(df['time'], utc=True)
    
    # Convertir a hora de España
    df['time_es'] = df['time'].dt.tz_convert(SPAIN_TZ)
    
    # Calcular precio medio de cada barra
    if 'high' in df.columns and 'low' in df.columns:
        df['price_avg'] = (df['high'] + df['low']) / 2
    else:
        # Si solo tienes bid/ask, usa (bid+ask)/2
        df['price_avg'] = (df['bid'] + df['ask']) / 2
    
    return df


In [ ]:
files = sorted([os.path.join(DATA_DIR, f) for f in os.listdir(DATA_DIR) if f.endswith('.xlsx')])

# Leer un día (por ejemplo el primero)
df_day = read_session_file(files[0])

df_day.head()


In [ ]:
plt.figure(figsize=(15,5))
plt.plot(df_day['time_es'], df_day['price_avg'], label='Precio medio (High-Low)')
plt.xlabel('Hora España')
plt.ylabel('Precio')
plt.title('Precio medio de barras SPY')
plt.legend()
plt.grid(True)
plt.show()
